In [ ]:
# !pip install python-dotenv
# !pip install langchain
# !pip install langchain-google-genai
# !pip install google-generativeai
# !pip install langgraph
# !pip install langchain-community
# !pip install duckduckgo_search
# !pip install tavily-python
# !pip install langgraph-checkpoint-sqlite

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
import getpass
from typing import TypedDict, Annotated
import operator
import re
from duckduckgo_search import DDGS
from bs4 import BeautifulSoup
import requests

from tavily import TavilyClient

from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.checkpoint.sqlite import SqliteSaver

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import ToolMessage

from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ConversationSummaryBufferMemory

from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.chains import ConversationChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

from langchain.prompts import PromptTemplate

from langgraph.graph import StateGraph, END


load_dotenv(find_dotenv())

False

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

Provide your Google API Key··········


In [ ]:
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Provide your Tavily API Key")

Provide your Tavily API Key··········


# Introduction

**Langchain** is a framework for developing applications powered by large language model. It is an open source ochestration framework for the development of applications that uses LLM

It is a generic interface for any LLM, abstracting the complexities of interacting with different LLMs

Chains, in langchain, are core of the langchain work flow. It combine LLMs with other components, creating applications by executing a sequence of functions

### Chat Models

**Chat models** is a language model that uses chat messages as input and return chat messages as outputs.

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("What is 2 + 2?")
print(f"Response: {result.content}")

Response: 4


## Prompt Template
It takes a prompt as input, which is set of instructions or input provided by user to guie model's response. It helps generate relevant output and understanding the context

**Prompt template** class formalizes the composition of prompts without the need to manually hard code context and queries

In [ ]:
template = """
Translate the text delimited by triple back ticks into
style that is {style}

text: ```{text}```
"""

In [ ]:
prompt_template = ChatPromptTemplate.from_template(template)
print(f"Prompt template passed to the model:\n{prompt_template.messages[0].prompt.template}")

Prompt template passed to the model:
 
Translate the text delimited by triple back ticks into
style that is {style}

text: ```{text}```



In [ ]:
print(f"Inputs in the prompt: {prompt_template.messages[0].prompt.input_variables}")

Inputs in the prompt: ['style', 'text']


In [ ]:
customer_email = "Yeh kya hae bhai? Yeh woh cheez nahi hae jo mein nein \
order ki thi. Tasweer mein yeh buhat muhktalif tha. Abb mein kya karoon? \
Replacment or return ki kya policy hae?"

style = "American style that is polite, calm and respectful in tone"

In [ ]:
customer_message = prompt_template.format_messages(style=style,
                                                   text=customer_email)

print(f"Customer message: {customer_message[0]}")

Customer message: content=' \nTranslate the text delimited by triple back ticks into\nstyle that is American style that is polite, calm and respectful in tone\n\ntext: ```Yeh kya hae bhai? Yeh woh cheez nahi hae jo mein nein order ki thi. Tasweer mein yeh buhat muhktalif tha. Abb mein kya karoon? Replacment or return ki kya policy hae?```\n'


In [ ]:
response = llm.invoke(customer_message)
print(f"Model response: {response.content}")

Model response: Excuse me, this is not what I ordered. It looks very different from the picture. What should I do now? What is your policy on replacements or returns?


In [ ]:
service_reply = """
Thanks for reaching out. We are sorry for inconvenience. \
We upload high quality pictures that best reflects the \
product quality. You can replace the item if it is ordered \
within 7 days. You cannot return the item. Our team will check \
the item and tell you what to do next. Thanks for understanding
"""

service_language = """
Translate the text into Urdu words using polite
tone.
"""

service_message = prompt_template.format_messages(style=service_language,
                                                  text=service_reply)


service_response = llm.invoke(service_message)
print(service_response.content)

شکریہ کہ ہمارے پاس پہنچے۔ ہم آپ کو ہونے والی پریشانی پر معذرت خواہ ہیں۔ ہم بہترین معیار کی تصاویر اپ لوڈ کرتے ہیں جو مصنوعات کے معیار کی عکاسی کرتی ہیں۔ اگر آپ نے 7 دنوں کے اندر آرڈر کیا ہے تو آپ اس آئٹم کو تبدیل کر سکتے ہیں۔ آپ اس آئٹم کو واپس نہیں کر سکتے۔ ہماری ٹیم آئٹم چیک کرے گی اور آپ کو بتائے گی کہ اس کے بعد کیا کرنا ہے۔ سمجھنے کے لئے شکریہ


You can see from the above example that instead of creating template for each response, we can create a template and reuse it

## Memory
Traditionally, large language models (LLMS) doesn't have any long term memory of prior conversation unless you pass the whole chat history as an input. Langchain solve this problem with simple utilities for adding in memory into your application

When we alk about memory in langchain, we are referring to how the system remember the information from previous interactions

In [ ]:
user = "Hello! My name is Maaz"
response = llm.invoke(user)
print(f"User: {user}\nResponse: {response.content}")

user = "What is 2 + 2?"
response = llm.invoke(user)
print(f"User: {user}\nResponse: {response.content}")

user = "What is my name?"
response = llm.invoke(user)
print(f"User: {user}\nResponse: {response.content}")

User: Hello! My name is Maaz
Response: Hello Maaz! It's great to meet you. I am Gemini, a multimodal AI language model. I am designed to understand and generate human language, and to answer your questions and assist you with a wide range of tasks. Is there anything I can help you with today?
User: What is 2 + 2?
Response: 4
User: What is my name?
Response: I do not have access to your personal information, including your name, as I am a chatbot assistant and do not have a physical presence or the ability to interact with the real world.


We can see that the model didn't remember my name despite being recently shared. The response of LLM model is stateless

### Conversation Buffer Memory

It stores the entire conversation history from beginning to end

In [ ]:
from langchain.memory import ConversationBufferMemory

llm = llm = ChatGoogleGenerativeAI(model="gemini-pro")

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

conversation.predict(input = "My name is Maaz")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My name is Maaz
AI:

> Finished chain.


'Hello Maaz, my name is Gemini. How can I help you today?'

In [ ]:
conversation.predict(input="What is 2 + 2?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Maaz
AI: Hello Maaz, my name is Gemini. How can I help you today?
Human: What is 2 + 2?
AI:

> Finished chain.


'2 + 2 is 4.'

In [ ]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Maaz
AI: Hello Maaz, my name is Gemini. How can I help you today?
Human: What is 2 + 2?
AI: 2 + 2 is 4.
Human: What is my name?
AI:

> Finished chain.


'Your name is Maaz, as you told me earlier.'

In [ ]:
print(f"Entire conversation: \n{memory.buffer}")

Entire conversation: 
Human: My name is Maaz
AI: Hello Maaz, my name is Gemini. How can I help you today?
Human: What is 2 + 2?
AI: 2 + 2 is 4.
Human: What is my name?
AI: Your name is Maaz, as you told me earlier.


In [ ]:
memory.load_memory_variables({})

{'history': 'Human: My name is Maaz\nAI: Hello Maaz, my name is Gemini. How can I help you today?\nHuman: What is 2 + 2?\nAI: 2 + 2 is 4.\nHuman: What is my name?\nAI: Your name is Maaz, as you told me earlier.'}

### Conversation Buffer Window Memory

It stores only the most recent exchanges within defined window size. Instead of saving entire conversation, it focuses on recent conversation

It is useful when the goal is to make AI focus on recent chat without overwheling by too much information

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1) #remember only 1 recent message

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

print(f"Conversations: \n{memory.buffer}")

Conversations: 
Human: Not much, just hanging
AI: Cool


In [ ]:

memory = ConversationBufferWindowMemory(k=2) #remember only 2 recent message

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "How is the weather"},
                    {"output": "Hot"})

print(f"Conversations: \n{memory.buffer}")

Conversations: 
Human: Not much, just hanging
AI: Cool
Human: How is the weather
AI: Hot


### Conversation Token Buffer Memory

Similar to the conversation buffer memory but instead of counting interactions, it count tokens

This memory type limits how much the AI remembers based on number of tokens. Once the token limit is reached, the oldest tokens are forgotten

In [ ]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=40)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})

print(f"Conversations: {memory.buffer}")

Conversations: Human: AI is what?!
AI: Amazing!
Human: Backpropagation is what?
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


In [ ]:
memory.save_context({"input": "How is the weather?"},
                    {"output": "Windy and cold!"})
print(f"Memory: \n{memory.buffer}")

Memory: 
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!
Human: How is the weather?
AI: Windy!
Human: How is the weather?
AI: Windy and cold!


### Conversation Summary Memory

It doesn't store every detail but creates a summary of past interactions. This allows the model to remember what is important without needing full conversation

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

print(f"Conversation in memory: \n{memory.buffer}")

Conversation in memory: 
System: The human says hello. The AI asks what's up. The human says they are hanging. The AI says that's cool. The human asks what is on the schedule today.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.


In [ ]:
conversation = ConversationChain(llm=llm,
                                 memory=memory,
                                 verbose=True)

conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human says hello. The AI asks what's up. The human says they are hanging. The AI says that's cool. The human asks what is on the schedule today.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'I recommend the latest demo for LLM that allows the model to generate text in the voice of a specific person, including celebrities.'

In [ ]:
print(f"New memory: {memory.buffer}")

New memory: System: The human says hello. The AI asks what's up. The human says they are hanging. The AI says that's cool. The human asks what is on the schedule today. The AI says there is a meeting at 8am with the product team, work on the LangChain project from 9am-12pm, and lunch at an Italian restaurant with a customer at Noon.
Human: What would be a good demo to show?
AI: I recommend the latest demo for LLM that allows the model to generate text in the voice of a specific person, including celebrities.


## Chains

In langchain, **chains** are essential worflows that connect different steps or components allowing to process information in structured way

Each step can involve tasks like

*  Making call to language model
*  Running a function
*  Making decision based on input


They are quite powerful because they enable us to build complex, multiple interaction with language model in a modular and reusable way

### Simple Sequential Chain
It is a straightforward type of chain that involves sequence of steps where output of one step become input of another

The chain executes each step in an order and doesn't involve amy branching or decision making

In [ ]:
from langchain.chains import SequentialChain

llm = ChatGoogleGenerativeAI(model="gemini-pro")
first_prompt = ChatPromptTemplate.from_template(
    "The email should be about {email_topic}?"
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

print(f"First chain: \n{first_chain}")
print(f"\nFirst chain input variables: \n{first_chain.prompt.input_variables}")
print(f"\nFirst chain input variables: \n{first_chain.prompt.messages}")

First chain: 
prompt=ChatPromptTemplate(input_variables=['email_topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['email_topic'], template='The email should be about {email_topic}?'))]) llm=ChatGoogleGenerativeAI(model='models/gemini-pro', client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f6f63bdf9a0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7f6f63bdf3d0>, default_metadata=())

First chain input variables: 
['email_topic']

First chain input variables: 
[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['email_topic'], template='The email should be about {email_topic}?'))]


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 word email on the email subject \
    {email_subject}"
)

second_chain = LLMChain(llm=llm,
                        prompt=second_prompt)

print(f"Second chain: \n{second_chain}")
print(f"\nSecond chain input variables: \n{second_chain.prompt.input_variables}")
print(f"\nSecond chain input variables: \n{second_chain.prompt.messages}")

Second chain: 
prompt=ChatPromptTemplate(input_variables=['email_subject'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['email_subject'], template='Write a 20 word email on the email subject     {email_subject}'))]) llm=ChatGoogleGenerativeAI(model='models/gemini-pro', client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f6f63bdf9a0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7f6f63bdf3d0>, default_metadata=())

Second chain input variables: 
['email_subject']

Second chain input variables: 
[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['email_subject'], template='Write a 20 word email on the email subject     {email_subject}'))]


In [ ]:
simple_chain = SimpleSequentialChain(chains=[first_chain, second_chain],
                                     verbose=False)

topic = 'Sick leave request to the manager'
print(f"Model response: \n{simple_chain.run(topic)}")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Model response: Subject: Sick Leave Request

Dear [Manager's Name],

I request sick leave from [start date] to [end date] due to [symptoms]. I've attached a doctor's note and arranged coverage with [coworker's name]. I'll stay updated and provide recovery updates.

Thanks,
[Your Name]


### Sequential Chain

**Sequential chain** is similar to simple sequential chain but more flexible and powerful.

It consists of multiple steps that are executed in a sequence. Each step can involve more complex operation such as combining output, branching, or integrating different type of models

Outout of one step is still passed to the next but can include different operations beyond just passing information

In [ ]:
from langchain.chains import SequentialChain

llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0)

first_prompt = ChatPromptTemplate.from_template(
    "Translate the review in English: {review}"
)
first_chain = LLMChain(llm=llm,
                       prompt=first_prompt,
                       output_key="english_review"
                       )

print(f"First Chain: \n{first_chain}")
print(f"First chain input variables: \n{first_chain.prompt.input_variables}")
print(f"First chain output variables: \n{first_chain.output_key}")

First Chain: 
prompt=ChatPromptTemplate(input_variables=['review'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], template='Translate the review in English: {review}'))]) llm=ChatGoogleGenerativeAI(model='models/gemini-pro', temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f6f60078340>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7f6f6007b6d0>, default_metadata=()) output_key='english_review'
First chain input variables: 
['review']
First chain output variables: 
english_review


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the review in 1 sentence: {english_review}"
)

second_chain = LLMChain(llm=llm,
                        prompt=second_prompt,
                        output_key="summary"
                        )

print(f"Second Chain: \n{second_chain}")
print(f"Second chain input variables: \n{second_chain.prompt.input_variables}")
print(f"Second chain output variables: \n{second_chain.output_key}")

Second Chain: 
prompt=ChatPromptTemplate(input_variables=['english_review'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['english_review'], template='Summarize the review in 1 sentence: {english_review}'))]) llm=ChatGoogleGenerativeAI(model='models/gemini-pro', temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f6f60078340>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7f6f6007b6d0>, default_metadata=()) output_key='summary'
Second chain input variables: 
['english_review']
Second chain output variables: 
summary


In [ ]:
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:{review}"
)
third_chain = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

print(f"Third Chain: \n{third_chain}")
print(f"Third chain input variables: \n{third_chain.prompt.input_variables}")
print(f"Third chain output variables: \n{third_chain.output_key}")

Third Chain: 
prompt=ChatPromptTemplate(input_variables=['review'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], template='What language is the following review:{review}'))]) llm=ChatGoogleGenerativeAI(model='models/gemini-pro', temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f6f60078340>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7f6f6007b6d0>, default_metadata=()) output_key='language'
Third chain input variables: 
['review']
Third chain output variables: 
language


In [ ]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
fourth_chain = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

sequential_chain = SequentialChain(chains=[first_chain, second_chain, third_chain, fourth_chain],
                                   input_variables=["review"],
                                   output_variables=["english_review", "summary", "followup_message"],
                                   verbose=True
                                   )

review = "Il fait très chaud, c'est très rapide, \
c'est très drôle mais le colis n'était pas bon ce \
qui a abîmé une partie"

response = sequential_chain(review)

print(f"Review: \n{response}")




> Entering new SequentialChain chain...

> Finished chain.
Review: 
{'review': "Il fait très chaud, c'est très rapide, c'est très drôle mais le colis n'était pas bon ce qui a abîmé une partie", 'english_review': 'It is very hot, it is very fast, it is very funny but the package was not good which damaged a part', 'summary': "Despite its impressive performance and humor, the product's packaging was inadequate, resulting in damage to a component.", 'followup_message': "Malgré ses performances impressionnantes et son humour, l'emballage du produit était inadéquat, ce qui a entraîné des dommages à un composant."}


### Router Chain

**Router chain** is an advanced type of chain that includes decision making. It routes input to different chains or steps based on certain conditions.

It is useful when different process has to be executed on type of user input or request

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.6)

# dictionary will store different LLM chains, each associated
# with specific prompt name
destination_chains = {}

# prompt info is list of dictionaries
for p_info in prompt_infos:
  # extract the value associated with the key 'name'
    name = p_info["name"]
  # extract value associated with the key 'prompt_template'
    prompt_template = p_info["prompt_template"]

    # creates a prompt template
    prompt = ChatPromptTemplate.from_template(template=prompt_template)

    # creates an instance of LLM chain and initialized with llm and prompt
    chain = LLMChain(llm=llm, prompt=prompt)

    # stores the chain in the destination chain using prompt name as key
    destination_chains[name] = chain

# list comprehension that iterates over each prompt info
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# creates a simple prompt template that directly passes the input to the
# language model without any special formatting
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
# creates final template that will be used to guide the routing process

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""


router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )


In [ ]:
chain.run("What is artifical intelligence?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is artifical intelligence?'}
> Finished chain.


'**Artificial Intelligence (AI)**\n\nArtificial Intelligence (AI) is a field of computer science that aims to create machines or systems that can perform tasks typically requiring human intelligence, such as learning, problem-solving, decision-making, and natural language processing.\n\n**Key Characteristics of AI:**\n\n* **Learning:** AI systems can learn from data and improve their performance over time, without explicit programming.\n* **Problem-Solving:** AI can analyze complex problems, identify patterns, and develop solutions.\n* **Decision-Making:** AI systems can make decisions based on available information and learned models.\n* **Natural Language Processing:** AI can understand, generate, and interpret human language effectively.\n* **Adaptability:** AI systems can adapt to changing environments and learn from new experiences.\n\n**Types of AI:**\n\n* **Machine Learning:** AI systems that learn from data without explicit programming.\n* **Deep Learning:** A subset of machine

In [ ]:
chain.run("What is 3 + 3?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 3 + 3?'}
> Finished chain.


'**Breaking down the problem:**\n\nThe problem "What is 3 + 3?" can be broken down into two component parts:\n\n1. What is 3?\n2. What is 3?\n\n**Answering the component parts:**\n\n1. 3 is a number.\n2. 3 is a number.\n\n**Putting the component parts together:**\n\nTo answer the broader question, we need to add the two numbers together:\n\n3 + 3 = 6\n\n**Therefore, the answer to the question "What is 3 + 3?" is 6.**'

## Agents

# LangGraph

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
result = llm.invoke("What is 2 + 2?")
print(f"Response: {result.content}")

Response: 2 + 2 = 4 



In [ ]:
class Agent:
  def __init__(self, system=""): # constructor that takes a system message
    self.system = system # system message is assigned to system variable
    self.messages=[] # empty list to track all messages exchanged during converstaion
    if self.system: # if system message is provided, it is added to the message list
      self.messages.append({"role": "system", "content": system})

  def __call__(self, message): # special method that allows an instance of Agent
                  # class to be called like a function
    self.messages.append({"role": "user", "content": message}) # the user message
                        # is appended to the message list-
    result = self.execute() # execute method used to get the response of message
    self.messages.append({"role": "assistant", "content": result}) # response is appended to message list
    return result

  def execute(self): # function that is called to execute the message list to get
                  # a response
    response = llm.invoke(self.messages)
    return response.content

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [ ]:
def calculate(attribute): # takes a string as input and uses the `eval()` function
              # to evaluate it as Python expression
    return eval(attribute) #eval function executes the attribute as python expression
          # if attribute = '5 + 2', the response will be 7

def average_dog_weight(name): # function that takes a name and return a reponse
                    # based on that name
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = { # maps strings to corresponding function objects. This allows
                  # for dynamic function calling
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [ ]:
agent = Agent(prompt) # prompt is passed as a system message
agent # you can see that the data type is Agent

In [ ]:
result = agent("How much does a Border Collie weigh?")
print(result)

Thought: I should look up the average weight of a Border Collie.
Action: average_dog_weight: Border Collie
PAUSE 



In [ ]:
result = average_dog_weight("Border Collie")
print(result)

a Border Collies average weight is 37 lbs


In [ ]:
next_prompt = f"Observation: {result}"
agent(next_prompt)
agent.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

Agents with Langgraph

In [ ]:
tool  = TavilySearchResults(max_results=2) # we will get only two responses from
                      # the search API
print(type(tool))
print(tool.name) # the name that the language model will use to call this tool

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [ ]:
class AgentState(TypedDict): # state of the agent that it will preserve throughout
                            # the workflow
  messages: Annotated[list[AnyMessage], operator.add] # Annotated allows the
  # developer to declare type of a reference and additional information related
  # to it

  # Annotated list of messages that we will add over time

In [ ]:
class Agent:
  # Three functions required.
  # 1. To call the language model
  # 2. Check whether an action is present
  # 3. To take that action

  def __init__(self, model, tools, system=""):
    self.system = system
    # creating the graph
    graph = StateGraph(AgentState)
    graph.add_node("llm",  self.call_llm) # add node to the graph and pass function that
                            # we want to represent this node
    graph.add_node("action", self.take_action)
    graph.add_conditional_edges(
        "llm", # where llm starts
        self.exists_action, # function that will determine where to go after that
        {True: "action", False: END} # dictionary representing how to map the response
                                  # of the fuction to the next node to go to
                                  # if the function returns true, we will go to
                                  # action node, else we will go to end node
    )
    graph.add_edge("action", "llm")
    graph.set_entry_point("llm")
    self.graph = graph.compile()
    self.tools = {t.name: t for t in tools} # create dictionary mapping the name
                                    # of the tool to the tool itself
    self.model = model.bind_tools(tools) # letting the model know that it has these
                              # tools available to call

  def exists_action(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0

  def call_llm(self, state: AgentState):
    messages = state['messages']
    if self.system:
      messages = [SystemMessage(content=self.system)] + messages
    message = self.model.invoke(messages)
    return {'messages': [message]}

  def take_action(self, state: AgentState): # action node
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
      print(f"Calling: {t}")
      result = self.tools[t['name']].invoke(t['args'])
      results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))

    print("Back to the model!")
    return {'messages': results}

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
agent = Agent(model, [tool], prompt)

In [ ]:
messages = [HumanMessage(content="What is the weather in Peshawar")]
result = agent.graph.invoke({"messages": messages})
print(result['messages'][-1].content)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Peshawar'}, 'id': 'ec18b40c-761b-48ce-a20e-9de9af04fe1b', 'type': 'tool_call'}
Back to the model!
The weather in Peshawar, Pakistan is currently 25.2 degrees Celsius and misty. The wind is blowing from the southeast at 15.1 kilometers per hour. The humidity is 83%. 



In [ ]:
messages = [HumanMessage(content="What is the weather in Peshawar and Taru Jabba?")]
result = agent.graph.invoke({"messages": messages})
print(result['messages'][-1].content)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Peshawar and Taru Jabba'}, 'id': 'eb38e6a2-dfb1-4923-846d-5a803943f796', 'type': 'tool_call'}
Back to the model!
I can't find the weather for Taru Jabba, but the weather in Peshawar is currently 10.9 degrees Celsius with patchy rain nearby. 



Agentic Search

In [ ]:
tavily_client = TavilyClient(api_key = os.environ.get("TAVILY_API_KEY"))
result = tavily_client.search('What is the weather in Madrid', include_answer = True)
print(result['answer'])

The current weather in Madrid is partly cloudy with a temperature of 30.3°C (86.5°F). The wind is coming from the NNE direction at 6.8 kph (4.3 mph), and the humidity is at 46%.


Simple searching

In [ ]:
city = "Madrid"

query = f"""
What is the current weather in {city}?
Should I travel there today for a picnic?
"""

In [ ]:
ddg = DDGS()

def search(query, max_results=3):
  results = ddg.text(query, max_results=max_results)
  return [i["href"] for i in results]

for i in search(query):
  print(i)

https://www.accuweather.com/en/es/madrid/308526/current-weather/308526
https://www.accuweather.com/en/es/madrid/308526/air-travel-weather/308526
https://www.accuweather.com/en/es/madrid/308526/weather-forecast/308526


In [ ]:
def scrape_weather_info(url):
  if not url:
    return "Weather information could not be found"

  headers = {"User-Agent": "Mozilla/5.0"}
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return "Failed to retrieve the webpage"

  soup = BeautifulSoup(response.text, "html.parser")
  return soup

url = search(query)[0]
soup = scrape_weather_info(url)

# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)

print(f"Website: {url}\n\n")
print(weather_data)

Website: https://www.accuweather.com/en/es/madrid/308526/current-weather/308526


Madrid, Madrid Madrid Madrid Around the Globe Around the Globe Hurricane Tracker Severe Weather Radar & Maps News & Features Astronomy Business Climate Health Recreation Sports Travel Video Current Weather 5:02 PM Day Yellow Warning for Storms 12:00 PM Friday - 12:00 AM Saturday Yellow Warning for Rain 12:00 PM Friday - 12:00 AM Saturday Max UV Index 5 Moderate Wind ENE 5 mph Wind Gusts 12 mph Probability of Precipitation 80% Probability of Thunderstorms 32% Precipitation 0.02 in Rain 0.02 in Hours of Precipitation 3 Hours of Rain 3 Cloud Cover 77% Morning Afternoon Night Yellow Warning for Storms 12:00 PM Friday - 12:00 AM Saturday Yellow Warning for Rain 12:00 PM Friday - 12:00 AM Saturday Wind ESE 5 mph Wind Gusts 14 mph Probability of Precipitation 40% Probability of Thunderstorms 24% Precipitation 0.05 in Rain 0.05 in Hours of Precipitation 1.5 Hours of Rain 1.5 Cloud Cover 38% Evening Overnight Sun 

Agentic Search

In [ ]:
result = tavily_client.search(query, max_results=1)
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'Madrid', 'region': 'Madrid', 'country': 'Spain', 'lat': 40.4, 'lon': -3.68, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1725030221, 'localtime': '2024-08-30 17:03'}, 'current': {'last_updated_epoch': 1725030000, 'last_updated': '2024-08-30 17:00', 'temp_c': 30.1, 'temp_f': 86.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 70, 'wind_dir': 'ENE', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 46, 'cloud': 50, 'feelslike_c': 28.8, 'feelslike_f': 83.9, 'windchill_c': 31.9, 'windchill_f': 89.5, 'heatindex_c': 30.9, 'heatindex_f': 87.6, 'dewpoint_c': 11.5, 'dewpoint_f': 52.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 8.0, 'gust_mph': 6.7, 'gust_kph': 10.8}}


In [ ]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "Madrid",
        "region": "Madrid",
        "country": "Spain",
        "lat": 40.4,
        "lon": -3.68,
        "tz_id": "Europe/Madrid",
        "localtime_epoch": 1725030221,
        "localtime": "2024-08-30 17:03"
    },
    "current": {
        "last_updated_epoch": 1725030000,
        "last_updated": "2024-08-30 17:00",
        "temp_c": 30.1,
        "temp_f": 86.2,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 2.5,
        "wind_kph": 4.0,
        "wind_degree": 70,
        "wind_dir": "ENE",
        "pressure_mb": 1015.0,
        "pressure_in": 29.97,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 46,
        "cloud": 50,
        "feelslike_c": 28.8,
        "feelslike_f": 83.9,
        "windchill_c": 31.9,
        "windchill_f": 89.5,
        "heatindex_

Persistence and Streaming in Agents

In [ ]:
tool = TavilySearchResults(max_results=2)
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
memory = SqliteSaver.from_conn_string(":memory:")

In [ ]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

ValidationError: 1 validation error for CompiledStateGraph
checkpointer
  instance of BaseCheckpointSaver expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCheckpointSaver)